In [1]:
import pandas as pd
import numpy as np

import pymysql
from sqlalchemy import create_engine

import getpass  # To get the password without showing the input

In [2]:
password = getpass.getpass()

········


In [3]:
user = 'root'  
host = 'localhost'  
database = 'sakila'  

connection_string = f"mysql+pymysql://{user}:{password}@{host}/{database}"

engine = create_engine(connection_string)

In [4]:
# Extract Relevant Information for Prediction Model (X)
X_query = """
SELECT 
    f.film_id,
    f.title,
    f.rental_rate,
    f.length,
    f.rating,
    COUNT(r.rental_id) AS rental_count
FROM 
    film f
LEFT JOIN inventory i ON f.film_id = i.film_id
LEFT JOIN rental r ON i.inventory_id = r.inventory_id
GROUP BY 
    f.film_id;
"""

X = pd.read_sql_query(X_query, engine)

# Create a Query for the Target Variable (y)
y_query = """
SELECT 
    f.film_id,
    MAX(CASE WHEN r.rental_date BETWEEN '2005-05-01' AND '2005-05-31' THEN 1 ELSE 0 END) AS rented_in_may
FROM 
    film f
LEFT JOIN inventory i ON f.film_id = i.film_id
LEFT JOIN rental r ON i.inventory_id = r.inventory_id
GROUP BY 
    f.film_id;
"""

y = pd.read_sql_query(y_query, engine)


In [5]:
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split

# Encoding categorical variables
encoder = OneHotEncoder()
X_encoded = encoder.fit_transform(X[['rating']]).toarray()

# Scaling numerical variables
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X[['rental_rate', 'length', 'rental_count']])

# Combine encoded and scaled features
X_combined = np.hstack((X_scaled, X_encoded))

In [6]:
from sklearn.linear_model import LogisticRegression

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_combined, y['rented_in_may'], test_size=0.3, random_state=42)

# Create and train the logistic regression model
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

In [7]:
from sklearn.metrics import accuracy_score, classification_report

y_pred = model.predict(X_test)

print(f"Accuracy: {accuracy_score(y_test, y_pred):.2f}")
print(classification_report(y_test, y_pred))

Accuracy: 0.74
              precision    recall  f1-score   support

           0       0.70      0.54      0.61       114
           1       0.75      0.86      0.80       186

    accuracy                           0.74       300
   macro avg       0.73      0.70      0.71       300
weighted avg       0.74      0.74      0.73       300



# Results

The model has a relatively high accuracy of 74%.
The model is better at predicting films that were rented.
The recall for films not rented is relatively low (0.54).
The precision for films not rented is also lower than for rented films, which means there are a fair number of false positives for the not rented category.